# 1: Introduction to 3D Vision

Follow the instructions in the [Introduction](../landing.py) to get started and come back here for some useful tips.

## Useful tips

Here are some useful tips to help you get started with the course.

### VS Code

I will be using Visual Studio Code for the course. You can download it [here](https://code.visualstudio.com/). 

#### Extensions

Here are some useful extensions that you can install:

- [Python](https://marketplace.visualstudio.com/items?itemName=ms-python.python)
- [Black Formatter](https://marketplace.visualstudio.com/items?itemName=ms-python.black-formatter)
  - Turn on `Format on Save`. (`Ctrl + ,`/`Cmd + ,` -> `format on save`)
- [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)

#### Shortcuts

- `Ctrl + P`/`Cmd + P`: Search for files
- `Ctrl + /`/`Cmd + /`: Comment/uncomment code
- `Ctrl + ,`/`Cmd + ,`: Open settings
- `Ctrl + C`/`Cmd + .`: Keyboard Interupt


### Type hinting

Type hinting is a way to specify the expected data types of function arguments and return values in Python. It helps improve code readability and maintainability by making the code more explicit and easier to understand. Take the following example:

```python
def add(a, b):
    return a + b
```

In this example, it is not clear what the expected types of `a` and `b` are, or what the return type of the function is. In fact, the function will work with any type that supports the `+` operator, which can lead to unexpected behavior.

For example if both `a` and `b` ints, the function will return an int:

```python
>>> a: int = 1
>>> b: int = 2
>>> result = add(a, b)
>>> type(result)
<class 'int'>
```

If we change `b` to a float, the function will return a float:

```python
>>> a: int = 1
>>> b: float = 2.0
>>> result = add(a, b)
>>> type(result)
<class 'float'>
```

This makes it hard for the user of the function to understand what input and output data to expect. 

#### How to use type hinting

By adding type hints to the function, we can make it clear what the expected types are:

```python
def add(a: int, b: int) -> int:
    return a + b
```

Now it is clear that the function expects two integers as input and will return an integer as output. This makes the code easier to understand and helps catch errors early on.

``` {tip}
In this repository typehinting is mandatory for all functions and classes, this is enforced by the use of `mypy` and `pyright` in the CI pipeline.
```

#### Numpy typing

When using numpy arrays, the `nptyping` library can be used to specify the expected shape and data type of the arrays. For example:

```python
from nptyping import Float32, NDArray, Shape

a: NDArray[Shape["3"], Float32] = ...
```

This specifies that `a` is a numpy array with shape `(3,)` and data type `float32`. The `Shape` can be used to specify a wide variety of shapes:

- **Numbers**: `Shape["3, 4"]` specifies a 3x4 matrix
- **Labels**: `Shape["H, W"]` specifies an image with height `H` and width `W`
- **Wildcard**: 
  - `Shape["*"]` specifies an one-dimensional array of any length.
  - `Shape["*, 3"]` specifies a two-dimensional array with 3 columns and any number of rows.
- **Ellipsis**: 
  - `Shape["..."]` specifies an array of any number of dimensions and any shape.
  - `Shape["3, ..."]` specifies an array with 3 elements in the first dimension and any number of elements in the remaining dimensions.

These can also be combined in various ways, for example `Shape["*, H, W, 3"]` can be used to specify a batch of images with height `H`, width `W`, and 3 channels (rgb). See the full documentation [here](https://github.com/ramonhagenaars/nptyping/blob/master/USERDOCS.md#syntax-shape-expressions).

### Jupyter Book

This project uses jupyter book to generate the documentation. The documentation is written in python, markdown and jupyter notebooks (.ipynb). The markdown files are rendered as is, but the jupyter notebooks are executed and the output is rendered in the documentation. This allows for a more interactive documentation experience. 

#### Python format

The python files are executed as notebooks using the `jupytext` library, with the `py:percent` format. This allows us to create a full python library with the benefits of `.py` files and at the same time be able to execute them as notebooks for the documentation. To write python code in the markdown files, you simply use `# %%` to define the cells:

##### Code Cell
    
```python
# %%
print("This is a code cell, everything inside will be executed as python code!)
```

##### Markdown Cell

```python
# %% [markdown]
#
# This is a markdown cell, it will not be executed as markdown code!
```

#### Interactive plots

When working in a jupyter notebook, a powerful feature is the ability to create interactive plots. Since the documentation is compiled to static html files, without a running python kernel, the interactive plots will not work. To work around this, we can use `bokeh` to create interactive plots using `javascript` callbacks. 

To use bokeh, we need to start the bokeh server in our code:

In [22]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import Slider, CustomJS, ColumnDataSource


output_notebook()

Loading BokehJS ...

In [24]:
import numpy as np


x = np.linspace(0, 1, 100) * 2 * np.pi
y = np.sin(2*x + np.pi)

source = ColumnDataSource(data=dict(x=x, y=y))
fig = figure(title="Interactive Sine Plot with Bokeh", x_axis_label='X', y_axis_label='Y')

fig.line('x', 'y', source=source, line_width=2)

slider_a = Slider(start=1, end=3, value=2, step=0.1, title="Amplitude (a)")
slider_b = Slider(start=0, end=2*np.pi, value=np.pi, step=np.pi/10, title="Phase shift (b)")

callback = CustomJS(args=dict(source=source, slider_a=slider_a, slider_b=slider_b), code="""
    const data = source.data;
    const a = slider_a.value;
    const b = slider_b.value;
    const x = data['x'];
    const y = data['y'];
    for (let i = 0; i < x.length; i++) {
        y[i] = Math.sin(a * x[i] + b);
    }
    source.change.emit();
""")

slider_a.js_on_change('value', callback)
slider_b.js_on_change('value', callback)

layout = column(fig, slider_a, slider_b)
show(layout)

``` {warning}
Sometimes you need to reload the page to see the interactive plots.
```